In [1]:
%load_ext autoreload
%autoreload 2
from importlib import reload
import glob
import pickle
import numpy as np
import os
import astropy.io.fits as fits
import pandas as pd
import pyklip.klip as klip
import pyklip.instruments.MagAO as MagAO
import pyklip.parallelized as parallelized
import pyklip.fakes as fakes
import matplotlib.pyplot as plt
import snr
import sys
import pdb
from tqdm import tqdm
import seaborn as sns
import collapse as c

#####################
# Display and backend
#####################
plt.style.use("default") # Reset to default before layering on any changes
FIG_LARGE = (12, 8) # Default plot figure size for large figures
%config InlineBackend.figure_format = "retina" # Crisp retina display on macs
#%matplotlib qt5 

################
# Theme settings
################
notebook_mode = "paper" # (paper | dark)
sns.set(palette="colorblind", color_codes=True, context="talk")

if notebook_mode.lower() == "paper":
    tick_params = {
        "xtick.top":True,
        "xtick.direction":"in",
        "ytick.right":True,
        "ytick.direction":"in",
        "axes.spine.right":True
    }
    sns.set_style("ticks", tick_params)
    params = {
        "axes.formatter.limits":(-3, 7),
        "xtick.major.size":14,
        "ytick.major.size":14,
        "xtick.minor.visible":True,
        "ytick.minor.visible":True,
    }
    plt.rcParams.update(params)

elif notebook_mode.lower() == "dark":
    plt.style.use("cyberpunk")

else:
    plt.style.use("default")

## Best Paramaters in for a KLIP reduction

In this notebook, we're going to explore the best parameters for a KLIP reduction for every dataset of interest. 

#### 1 ) Find Params

In [3]:
filepaths = ['pe_falseplanets/paramexplore_HD142527_11Apr13_Cont_10pctcut_FAKES_a1-25x1m0-24x1s1iwa3_hp1.5-KLmodes-all.fits', 'pe_falseplanets/paramexplore_HD142527_8Apr14_Cont_0pctcut_FAKES_a1-25x1m0-24x1s1iwa4_hp2.0-KLmodes-all.fits', 'pe_falseplanets/paramexplore_HD142527_15May15_Cont_50pctcut_FAKES_a1-25x1m0-24x1s1iwa4_hp2.0-KLmodes-all.fits', 'pe_falseplanets/paramexplore_HD142527_16May15_Cont_80pctcut_FAKES_a1-25x1m0-24x1s1iwa4_hp2.0-KLmodes-all.fits', 'pe_falseplanets/paramexplore_HD142527_27Apr18_Cont_0pctcut_FAKES_a1-25x1m0-24x1s1iwa3_hp1.5-KLmodes-all.fits']
ras = [11.3, 9.8, 9.1, 9.1, 5.8]
pas = [125, 113, 110, 110, 55]
highpasses = [1.5, 2, 2, 1.5, 2]
datanames = ['HD142527 11Apr13', 'HD142527 8Apr14', 'HD142527 15May15', 'HD142527 16May15', 'HD142527 27Apr18']
results_mean = []

for filepath, ra, pa, highpass, dataname in zip(filepaths, ras, pas, highpasses, datanames):
    collapser = c.PECollapser(filepath, handle_planets='mean', dataname = dataname, ra = ra, pa = pa, hp = highpass)
    result = pd.DataFrame(collapser.collapse_across_planets(), index = [0])
    results_mean.append(result)


all_params_mean = pd.concat(results_mean)


#### Plot

In [4]:
filepaths = ['pe_falseplanets/paramexplore_HD142527_11Apr13_Cont_10pctcut_FAKES_a1-25x1m0-24x1s1iwa3_hp1.5-KLmodes-all.fits', 'pe_falseplanets/paramexplore_HD142527_8Apr14_Cont_0pctcut_FAKES_a1-25x1m0-24x1s1iwa4_hp2.0-KLmodes-all.fits', 'pe_falseplanets/paramexplore_HD142527_15May15_Cont_50pctcut_FAKES_a1-25x1m0-24x1s1iwa4_hp2.0-KLmodes-all.fits', 'pe_falseplanets/paramexplore_HD142527_16May15_Cont_80pctcut_FAKES_a1-25x1m0-24x1s1iwa4_hp2.0-KLmodes-all.fits', 'pe_falseplanets/paramexplore_HD142527_27Apr18_Cont_0pctcut_FAKES_a1-25x1m0-24x1s1iwa3_hp1.5-KLmodes-all.fits']
ras = [11.3, 9.8, 9.1, 9.1, 5.8]
pas = [125, 113, 110, 110, 55]
highpasses = [1.5, 2, 2, 1.5, 2]
datanames = ['HD142527 11Apr13', 'HD142527 8Apr14', 'HD142527 15May15', 'HD142527 16May15', 'HD142527 27Apr18']
results_median = []

for filepath, ra, pa, highpass, dataname in zip(filepaths, ras, pas, highpasses, datanames):
    collapser = c.PECollapser(filepath, handle_planets='median', dataname = dataname, ra = ra, pa = pa, hp = highpass)
    result = pd.DataFrame(collapser.collapse_across_planets(), index = [0])
    results_median.append(result)


all_params_median = pd.concat(results_median)


In [5]:
## Make KL string list and then pull the index for table display
all_params_mean

,Dataname,RA,PA,highpass,PeakSNR_annuli,PeakSNR_movement,PeakSNR_kl,AvgSNR_annuli,AvgSNR_movement,AvgSNR_kl,Contrast_annuli,Contrast_movement,Contrast_kl,Avg_annuli,Avg_move,Avg_kl
0,HD142527 11Apr13,11.3,125,1.5,14,1,6,15,1,7,25,0,4,14,1,6
0,HD142527 8Apr14,9.8,113,2.0,24,2,6,25,2,5,22,1,7,24,2,7
0,HD142527 15May15,9.1,110,2.0,16,2,1,16,1,5,9,1,7,16,3,1
0,HD142527 16May15,9.1,110,1.5,1,0,2,2,5,0,13,1,7,1,0,2
0,HD142527 27Apr18,5.8,55,2.0,24,2,6,23,2,5,11,1,6,25,2,7


In [6]:
all_params_median

,Dataname,RA,PA,highpass,PeakSNR_annuli,PeakSNR_movement,PeakSNR_kl,AvgSNR_annuli,AvgSNR_movement,AvgSNR_kl,Contrast_annuli,Contrast_movement,Contrast_kl,Avg_annuli,Avg_move,Avg_kl
0,HD142527 11Apr13,11.3,125,1.5,22,1,5,25,1,7,25,0,4,25,1,7
0,HD142527 8Apr14,9.8,113,2.0,24,1,6,22,2,6,22,1,7,24,1,6
0,HD142527 15May15,9.1,110,2.0,23,3,1,22,1,6,9,1,7,23,3,1
0,HD142527 16May15,9.1,110,1.5,1,0,4,18,2,3,13,1,7,1,0,4
0,HD142527 27Apr18,5.8,55,2.0,20,2,4,1,15,6,11,1,6,23,1,6
